# Análise das principais características socioeconômicas que se relacionam com a média das notas dos alunos no ENEM: uma abordagem utilizando técnicas de ciência de dados e redução de dimensionalidade nas bases de dados de 2018 à 2022.

## Introdução

... Contextualizar sobre o ENEM
... Falar sobre a alta dimencionalidade da base
... Falar sobre o objetivo

## Importação dos dados e recursos necessários

In [1]:
# !pip install category_encoders

In [2]:
import pandas as pd
import category_encoders as ce

from category_encoders.one_hot import OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, f_classif
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression

In [3]:
DATASET_ENEM_PATH = 'D:\Rafael\Projects\IFMG_TCC_ENEM\Datasets\MICRODADOS_ENEM_2021.csv'

In [4]:
microdadosEnem = pd.read_csv(DATASET_ENEM_PATH, sep=';', encoding='ISO-8859-1')

## Análise exploratória dos dados.

In [5]:
microdadosEnem.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B


In [6]:
microdadosEnem.describe()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q005
count,3.389832e+06,3389832.0,3.389832e+06,3.389832e+06,3.389832e+06,3.389832e+06,3.389832e+06,3.389832e+06,3.389832e+06,1.096828e+06,...,2.245844e+06,3.389832e+06,2.378379e+06,2.378379e+06,2.378379e+06,2.378379e+06,2.378379e+06,2.378379e+06,2.378379e+06,3.389830e+06
mean,2.100527e+11,2021.0,5.306013e+00,1.052073e+00,2.043491e+00,1.027106e+00,1.608430e+00,2.581980e+00,1.396214e+00,1.006295e+00,...,5.350802e+02,4.704640e-01,1.140684e+00,1.192657e+02,1.380269e+02,1.153117e+02,1.281853e+02,1.127109e+02,6.135005e+02,3.739557e+00
std,9.962403e+05,0.0,3.902871e+00,4.129414e-01,1.022561e+00,2.049293e-01,7.198306e-01,4.278731e+00,5.938455e-01,7.909361e-02,...,1.106267e+02,4.991269e-01,7.910804e-01,3.391962e+01,4.563224e+01,3.634138e+01,3.883252e+01,6.005667e+01,1.907142e+02,1.378942e+00
min,2.100510e+11,2021.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.100519e+11,2021.0,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,4.447000e+02,0.000000e+00,1.000000e+00,1.000000e+02,1.200000e+02,1.000000e+02,1.200000e+02,8.000000e+01,5.200000e+02,3.000000e+00
50%,2.100527e+11,2021.0,4.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,5.154000e+02,0.000000e+00,1.000000e+00,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,6.000000e+02,4.000000e+00
75%,2.100536e+11,2021.0,7.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,...,6.139000e+02,1.000000e+00,1.000000e+00,1.400000e+02,1.800000e+02,1.400000e+02,1.600000e+02,1.600000e+02,7.400000e+02,4.000000e+00
max,2.100545e+11,2021.0,2.000000e+01,4.000000e+00,6.000000e+00,4.000000e+00,4.000000e+00,1.500000e+01,3.000000e+00,2.000000e+00,...,9.531000e+02,1.000000e+00,9.000000e+00,2.000000e+02,2.000000e+02,2.000000e+02,2.000000e+02,2.000000e+02,1.000000e+03,2.000000e+01


In [7]:
microdadosEnem.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
       'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
       'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', '

Como o objetivo principal do projeto é analisar as principais características socioeconômicas que influenciam a média de notas dos alunos no ENEM, é necessário prestar atenção especial às colunas: "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO". De acordo com o dicionário de dados, essas colunas representam as notas obtidas em cada área do conhecimento. Desta forma, estas colunas serão chamadas de "target". Em linhas gerais, serão evidenciados quais padrões as demais colunas seguem para amplificar ou reduzir os valores das colunas target. Adiante, simplificaremos todas as colunas target em uma coluna de média

In [8]:
colunasTarget = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

## Eliminando características individuais, de baixa variância ou inexpressivas.

De forma intuitiva, é possível constatar que as colunas "NU_INSCRICAO", "NU_ANO", "TX_RESPOSTAS_CN", "TX_RESPOSTAS_CH", "TX_RESPOSTAS_LC", "TX_RESPOSTAS_MT", "TX_GABARITO_CN", "TX_GABARITO_CH", "TX_GABARITO_LC", "TX_GABARITO_MT", "CO_PROVA_CN", "CO_PROVA_CH", "CO_PROVA_LC" e "CO_PROVA_MT" possuem pouca relevância para o objetivo da análise. Por meio do dicionário de dados, sabe-se que essas colunas representam, respectivamente: o número de inscrição do candidato, o ano de realização da prova, cada um os vetores de respostas objetivas para as diferentes áreas de conhecimento da prova e os códigos referêntes as cores de prova que cada candidato recebeu.

Dada a devida contextualização sobre o propósito das colunas, podemos inferir que a coluna "NU_ANO" possui variância zero, dado que o conjunto de dados em questão contém apenas registros do ano de 2021. As colunas: "NU_INSCRICAO", "TX_RESPOSTAS_CN", "TX_RESPOSTAS_CH", "TX_RESPOSTAS_LC", "TX_RESPOSTAS_MT", "TX_GABARITO_CN", "TX_GABARITO_CH", "TX_GABARITO_LC" e "TX_GABARITO_MT", por sua vez, têm variância muito alta. Estas colunas representam caracteristicas altamente individuais dos candidatos e, portanto, não são aptas para avaliação de nenhum tipo de tendência. Por fim, as colunas: "CO_PROVA_CN", "CO_PROVA_CH", "CO_PROVA_LC" e "CO_PROVA_MT" são inexpressivas dado que as variantes de uma mesma prova são elaboradas mantendo o mesmo nível de dificuldade.

Validação de hipótese: 

In [9]:
colunasParaRemocao = [
    'NU_INSCRICAO', 'NU_ANO', 'TX_RESPOSTAS_CN',
    'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
    'TX_GABARITO_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
    'CO_PROVA_MT'
]

microdadosEnem.drop(columns=colunasParaRemocao, inplace=True)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,5,F,1,1,1,1,3,1,NaN,0,...,A,A,A,B,A,A,B,A,B,B
1,12,M,1,1,1,1,11,1,NaN,0,...,A,A,A,B,A,A,C,A,A,A
2,13,F,3,1,1,1,15,1,NaN,0,...,B,A,A,B,A,A,C,B,B,B
3,3,M,1,3,1,2,0,2,1.0,0,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,0,2,1.0,0,...,B,A,A,B,A,B,E,A,B,B


## Eliminando redundâncias

Além disso, iremos eliminar colunas redundantes, por exemplo: "NO_MUNICIPIO_RESIDENCIA" (nome do município) equivalente a "CO_MUNICIPIO_RESIDENCIA" (código do município).

In [10]:
colunasParaRemocao = [
    'NO_MUNICIPIO_ESC','CO_UF_ESC','SG_UF_ESC', # == CO_MUNICIPIO_ESC
    'NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA', # == CO_MUNICIPIO_PROVA
]

In [11]:
microdadosEnem.drop(columns=colunasParaRemocao, inplace=True)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,5,F,1,1,1,1,3,1,NaN,0,...,A,A,A,B,A,A,B,A,B,B
1,12,M,1,1,1,1,11,1,NaN,0,...,A,A,A,B,A,A,C,A,A,A
2,13,F,3,1,1,1,15,1,NaN,0,...,B,A,A,B,A,A,C,B,B,B
3,3,M,1,3,1,2,0,2,1.0,0,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,0,2,1.0,0,...,B,A,A,B,A,B,E,A,B,B


## Tratamento de dados faltantes

Uma abordagem possível para lidar com dados faltantes seria remover do dataset todos os registros que possuem alguma coluna não preenchida. Ao seguir esse caminho, estaríamos mantendo apenas os dados mais consistentes para as análises posteriores. No entanto, é importante observar que essa abordagem resultaria na perda de aproximadamente 83% dos registros em nossa base de dados atual. Essa perda representa uma quantidade significativa de informações e não é aceitável em termos de tolerância. Segue abaixo a comprovação.

In [12]:
totalLinhasAntesDrop = microdadosEnem.shape[0]
totalLinhasDepoisDrop = microdadosEnem.dropna().shape[0]

print("Linhas restantes: ", totalLinhasAntesDrop - totalLinhasDepoisDrop)
print("Percentual de linhas mantidas: ", (totalLinhasDepoisDrop / totalLinhasAntesDrop) * 100)

Linhas restantes:  2797643
Percentual de linhas mantidas:  17.469567813390164


### Tratando os dados faltantes caso a caso.

Dado que a remoção de todos os registros de qualquer participante que possui qualquer coluna não preenchida se mostrou um caminho inviável devido à perda excessiva de dados, será necessário adotar outra abordagem para lidar com os valores vazios. Desta forma, será necessário identificar quais colunas possuem valores nulos e buscar, em cada caso, uma estratégia menos impactante para lidar com esses valores.

Para facilitar a investigação sobre quais colunas apresentam os maiores problemas de ausência de dados, as colunas remanescentes foram divididas em grupos. Essa divisão tem o único objetivo de melhorar a visualização dos dados.

In [13]:
microdadosEnem.columns.values

array(['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
       'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'CO_MUNICIPIO_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
       'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH',
       'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004',
       'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012',
       'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020',
       'Q021', 'Q022', 'Q023', 'Q024', 'Q025'], dtype=object)

In [14]:
colunas_grupo1 = [
    'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL',
    'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
    'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
    'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
    'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
    'TP_PRESENCA_MT', 'NU_NOTA_CN',
    'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
    'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'
]

colunas_grupo2 = [
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
]

In [15]:
microdadosEnem[colunas_grupo1].isna().sum()

TP_FAIXA_ETARIA                 0
TP_SEXO                         0
TP_ESTADO_CIVIL                 0
TP_COR_RACA                     0
TP_NACIONALIDADE                0
TP_ST_CONCLUSAO                 0
TP_ANO_CONCLUIU                 0
TP_ESCOLA                       0
TP_ENSINO                 2293004
IN_TREINEIRO                    0
CO_MUNICIPIO_ESC          2576026
TP_DEPENDENCIA_ADM_ESC    2576026
TP_LOCALIZACAO_ESC        2576026
TP_SIT_FUNC_ESC           2576026
CO_MUNICIPIO_PROVA              0
TP_PRESENCA_CN                  0
TP_PRESENCA_CH                  0
TP_PRESENCA_LC                  0
TP_PRESENCA_MT                  0
NU_NOTA_CN                1143988
NU_NOTA_CH                1011453
NU_NOTA_LC                1011453
NU_NOTA_MT                1143988
TP_LINGUA                       0
TP_STATUS_REDACAO         1011453
NU_NOTA_COMP1             1011453
NU_NOTA_COMP2             1011453
NU_NOTA_COMP3             1011453
NU_NOTA_COMP4             1011453
NU_NOTA_COMP5 

In [16]:
microdadosEnem[colunas_grupo2].isna().sum()

Q001    2
Q002    2
Q003    2
Q004    2
Q005    2
Q006    2
Q007    2
Q008    2
Q009    2
Q010    2
Q011    2
Q012    2
Q013    2
Q014    2
Q015    2
Q016    2
Q017    2
Q018    2
Q019    2
Q020    2
Q021    2
Q022    2
Q023    2
Q024    2
Q025    2
dtype: int64

#### LEVANTAMENTOS:

1. No primeiro grupo de colunas, observamos uma quantidade significativa de registros com informações ausentes relacionadas à escola do candidato. São elas:

    * "CO_MUNICIPIO_ESC" (Código do município da escola);
    * "TP_DEPENDENCIA_ADM_ESC" (Dependência administrativa da escola);
    * "TP_LOCALIZACAO_ESC" (Tipo de localização da escola); 
    * "TP_SIT_FUNC_ESC" (Situação de funcionamento da escola);
    * "TP_ENSINO" (Tipo de instituição de ensino);
    
Dado que aproximadamente 75% das linhas da tabela não possuem esses valores preenchidos, optaremos por remover essas colunas do modelo, pois não será possivel utiliza-las como objeto de ánalise confiável e nem fazer nenhum outro tipo de inferência preditiva.

In [17]:
colunasParaRemocao = [
  'CO_MUNICIPIO_ESC','TP_DEPENDENCIA_ADM_ESC','TP_LOCALIZACAO_ESC',
  'TP_SIT_FUNC_ESC', 'TP_ENSINO'
]

In [18]:
microdadosEnem.drop(columns=colunasParaRemocao, inplace=True)

2. Como dito anterirmente, o objetivo principal da análise é encontrar relações entre os dados socioeconômicos dos participantes com sua média de notas. Por meio do dicionário de dados, sabe-se que as colunas responsáveis por indicar a nota dos candidatos são as colunas "_target_". Portanto, todos os registros de alunos com qualquer uma dessas colunas não preenchidas devem ser desconsiderados da análise, uma vez que seus dados socioeconômicos não podem ser confrontados com as notas obtidas.

In [19]:
microdadosEnem.dropna(subset=colunasTarget, inplace=True)

3. Poucos registros não possuem os campos de Q001 a Q025 preenchidos. Esses registros serão desconsiderados. _(OBS: Avaliar depois a possiblidade de aplicar uma técnica de preechimento baseada na técnica do mais frequente para preservar estas duas linhas)_

In [20]:
microdadosEnem.dropna(subset=colunas_grupo2, inplace=True)

In [21]:
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1,12,M,1,1,1,1,11,1,0,2704302,...,A,A,A,B,A,A,C,A,A,A
3,3,M,1,3,1,2,0,2,0,2304202,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,0,2,0,2300150,...,B,A,A,B,A,B,E,A,B,B
8,8,F,1,3,1,2,0,2,0,3106200,...,B,A,A,B,A,A,B,A,A,B
9,4,F,1,3,1,2,0,2,0,2927200,...,B,A,B,C,B,A,C,A,A,B


#### REAVALIAÇÃO DA CONSISTÊNCIA DOS DADOS:

In [22]:
microdadosEnem.isna().sum().sum()

0

In [23]:
microdadosEnem.shape

(2238106, 51)

#### ANÁLISE DOS GANHOS

Seguindo esta abordagem, foram resolvidos todos dos problemas com dados nulos e mantivemos 2.238.106 registros para realização das próximas analises. 2.797.643

In [24]:
print("Nº de linhas mantidas em caso de dropna sem tratamento: ", totalLinhasDepoisDrop)
print("Nº de linhas mantidas em caso de tratamento: ", microdadosEnem.shape[0])
print("Nº de linhas que foram preservadas com o tratamento: ", microdadosEnem.shape[0] - totalLinhasDepoisDrop)
print("Percentual de linhas mantidas da tabela original: ", (microdadosEnem.shape[0] / totalLinhasAntesDrop) * 100)

Nº de linhas mantidas em caso de dropna sem tratamento:  592189
Nº de linhas mantidas em caso de tratamento:  2238106
Nº de linhas que foram preservadas com o tratamento:  1645917
Percentual de linhas mantidas da tabela original:  66.0240979493969


#### LEVANTAMENTO DE HIPÓTESE - NOVA ANÁLISE DE VARIÂNCIA 

Levanta-se a hipotese de que as colunas indicativas da presença do participante em cada uma das provas deverá estar com variância nula, dado que foram removidos os participantes ausêntes por outro método. 

In [25]:
colunasParaRemocao = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']

In [26]:
microdadosEnem[colunasParaRemocao].var()

TP_PRESENCA_CN    0.0
TP_PRESENCA_CH    0.0
TP_PRESENCA_LC    0.0
TP_PRESENCA_MT    0.0
dtype: float64

Dado que a hipotese foi validada:

In [27]:
microdadosEnem.drop(columns=colunasParaRemocao, inplace=True)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1,12,M,1,1,1,1,11,1,0,2704302,...,A,A,A,B,A,A,C,A,A,A
3,3,M,1,3,1,2,0,2,0,2304202,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,0,2,0,2300150,...,B,A,A,B,A,B,E,A,B,B
8,8,F,1,3,1,2,0,2,0,3106200,...,B,A,A,B,A,A,B,A,A,B
9,4,F,1,3,1,2,0,2,0,2927200,...,B,A,B,C,B,A,C,A,A,B


## Tratando variáveis categóricas

... Add descrição

In [28]:
microdadosEnem.select_dtypes(include='object').describe()

,TP_SEXO,Q001,Q002,Q003,Q004,Q006,Q007,Q008,Q009,Q010,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
count,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106,...,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106,2238106
unique,2,8,8,6,6,17,4,5,5,5,...,5,5,2,5,2,2,5,2,5,2
top,F,E,E,D,B,B,A,B,C,A,...,B,A,A,B,A,A,D,A,B,B
freq,1381948,681530,795005,522352,863295,546733,2022176,1343008,1061859,1033708,...,1205352,2154648,1617349,1384848,1754598,1673806,714811,1841982,954011,2048739


#### Aplicação da técnica One-Hot Encoding para tratar variáveis categóricas nominais

In [29]:
oneHotEncoder = OneHotEncoder(cols=['TP_SEXO'])
microdadosEnem = oneHotEncoder.fit_transform(microdadosEnem)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO_1,TP_SEXO_2,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1,12,1,0,1,1,1,1,11,1,0,...,A,A,A,B,A,A,C,A,A,A
3,3,1,0,1,3,1,2,0,2,0,...,A,A,A,B,A,A,B,A,B,B
4,2,0,1,1,3,1,2,0,2,0,...,B,A,A,B,A,B,E,A,B,B
8,8,0,1,1,3,1,2,0,2,0,...,B,A,A,B,A,A,B,A,A,B
9,4,0,1,1,3,1,2,0,2,0,...,B,A,B,C,B,A,C,A,A,B


#### Aplicação da técnica Ordinal Encoding para tratar variáveis categóricas ordinais

In [30]:
ordinalEncoder = OrdinalEncoder(cols=[
    'Q001', 'Q002', 'Q003', 'Q004', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
])

microdadosEnem = ordinalEncoder.fit_transform(microdadosEnem)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO_1,TP_SEXO_2,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1,12,1,0,1,1,1,1,11,1,0,...,1,1,1,1,1,1,1,1,1,1
3,3,1,0,1,3,1,2,0,2,0,...,1,1,1,1,1,1,2,1,2,2
4,2,0,1,1,3,1,2,0,2,0,...,2,1,1,1,1,2,3,1,2,2
8,8,0,1,1,3,1,2,0,2,0,...,2,1,1,1,1,1,2,1,1,2
9,4,0,1,1,3,1,2,0,2,0,...,2,1,2,2,2,1,1,1,1,2


### Separação entre os dados alvo e as caracteristicas

In [31]:
microdadosEnem.columns.values

array(['TP_FAIXA_ETARIA', 'TP_SEXO_1', 'TP_SEXO_2', 'TP_ESTADO_CIVIL',
       'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO',
       'CO_MUNICIPIO_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
       'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
       'Q023', 'Q024', 'Q025'], dtype=object)

In [33]:
colunasFeatures = [
    'TP_FAIXA_ETARIA', 'TP_SEXO_1', 'TP_SEXO_2', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
    'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
    'TP_ESCOLA', 'IN_TREINEIRO', 'CO_MUNICIPIO_PROVA', 'TP_LINGUA',
    'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
    'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
]

microdadosEnem_baseTarget = pd.DataFrame()
microdadosEnem_baseTarget['MEDIA'] = (microdadosEnem.NU_NOTA_CN + microdadosEnem.NU_NOTA_CH+ microdadosEnem.NU_NOTA_LC + microdadosEnem.NU_NOTA_MT + microdadosEnem.NU_NOTA_REDACAO)/5

microdadosEnem_baseFeatures = microdadosEnem[colunasFeatures]

In [34]:
microdadosEnem_baseTarget.head()

,MEDIA
1,515.50
3,667.60
4,570.06
8,485.60
9,466.94


In [35]:
microdadosEnem_baseFeatures.head()

,TP_FAIXA_ETARIA,TP_SEXO_1,TP_SEXO_2,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1,12,1,0,1,1,1,1,11,1,0,...,1,1,1,1,1,1,1,1,1,1
3,3,1,0,1,3,1,2,0,2,0,...,1,1,1,1,1,1,2,1,2,2
4,2,0,1,1,3,1,2,0,2,0,...,2,1,1,1,1,2,3,1,2,2
8,8,0,1,1,3,1,2,0,2,0,...,2,1,1,1,1,1,2,1,1,2
9,4,0,1,1,3,1,2,0,2,0,...,2,1,2,2,2,1,1,1,1,2


## Seleção automática de features: SelectKBest + f_regression

TODO:: Justificar a escolha deste método

In [43]:
microdadosEnem_baseFeatures.shape

(2238106, 43)

In [37]:
microdadosEnem_baseTarget.shape

(2238106, 1)

In [38]:
feature_selector = SelectKBest(score_func = f_regression, k = 20)
melhoresFeatures_metodo1 = feature_selector.fit_transform(microdadosEnem_baseFeatures, microdadosEnem_baseTarget.MEDIA.values)

In [39]:
print("Número original de caracteristicas: ", microdadosEnem_baseFeatures.shape[1])
print("Número reduzido de caracteristicas: ", melhoresFeatures_metodo1.shape[1])
print("Quais as caracteristicas selecionadas por este método: ", microdadosEnem_baseFeatures.columns[feature_selector.get_support()])

Número original de caracteristicas:  43
Número reduzido de caracteristicas:  20
Quais as caracteristicas selecionadas por este método:  Index(['TP_COR_RACA', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'Q006', 'Q007', 'Q008', 'Q010', 'Q013', 'Q014', 'Q016', 'Q018', 'Q021',
       'Q023', 'Q024', 'Q025'],
      dtype='object')


TODO:: Aplicar outros métodos

TODO:: Comparar resultados dos métodos aplicados e selecionar apenas caracteristicas recorrentes

TODO:: Comparar as caracteristicas recorrentes deste ano com as caracteristicas recorrentes de todos os outros anos